# Basic of Text Mining with R

In [1]:
# install.packages(c('rvest', 'httr', 'stringr', 'readxl', 'dplyr', 'tm', 'qgraph', 'KoNLP'), repos='http://cran.nexr.com')

### Internet 
 - Client -> http -> Server (Request)
 - Server -> http -> Client (Response)

### Get & Post 
 - 200 OK -> 서버가 내가 원하는 코드를 주는 것이다. 
 - 그외의 것은 서버가 고장났거나, 내가 이상하게 보냈거나.

### 크롤링을 하는 이유
 - 다양한데이터가 인터넷에 존재한다. 쉽고 빠르게 정보를 획득 할 수 있다. 
 - 내용을 분석하기 위해서 크롤링한다. 내가 필요한 부분만 빼낸다. 

In [2]:
# <p>안녕하세요\</p> P태그로 감싸져있다. 
#  - Element 라고도 하고 Node라고도한다. (엄격하게는 다르다.)
# <h3 class='title'> 텍스트에 감정읽기 </h3>  // class 는 여러개를 가질 수 있다. 
# <h3 id='title'> 텍스트에 감정읽기 </h3>    // id는 하나밖에 없다. 페이지하나에.

In [1]:
library(httr) # http 통신.
#url = 'http://movie.daum.net/moviedetail/moviedetailNetizenPoint.do?movieId=73750&t__nil_NetizenPoint=tabName'

Warning message:
: package 'httr' was built under R version 3.3.2

In [2]:
url = "http://movie.daum.net/moviedb/grade?movieId=54081&type=netizen&page=5"
r <- GET(url)

In [3]:
r

Response [http://movie.daum.net/moviedb/grade?movieId=54081&type=netizen&page=5]
  Date: 2017-03-22 13:47
  Status: 200
  Content-Type: text/html;charset=UTF-8
  Size: 88.1 kB










...

In [5]:
library(rvest)
library(dplyr)

In [6]:
h = read_html(r) # Read HTML or XML.
comment_area = html_nodes(h, '.list_review') # list_review안에 다 있다.    // .은 class

In [7]:
comments = html_nodes(comment_area, '.desc_review') # 그 안에서 desc_review안에 코멘트가 존재한다. id는 #
comments = repair_encoding(html_text(comments)) # html_text 노드사이에있는 텍스트를 가지고 온다. 
comments

Best guess: UTF-8 (100% confident)


[1] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                               
 [2] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                               
 [3] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                               
 [4] " \n\t\t\t\t\t\t\t\t\t\t통쾌한 액션과 여러 히어로들의 총출동. 그래도 유치하다\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                          
 [5] " \n\t\t\t\t\t\t\t\t\t\t누구보다 빠르게 남들과는 다르게 씽나게 봤음\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                    
 [6] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                               
 [7] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                               
 [8] " \n\t\t\t\t\t\t\t\t\t\t!!!!!!!\n\t\t\t\t\t\t\t\t\t"                                                                                                                                                                                                                                                                                        
 [9] " \n\t\t\t\t\t\t\t\t\t\t단합이 안되던 어벤져스가 어셈블을 이뤄 인류정복을 꿈꾸는 교만한 로키의 카리스마를 박살낸다. 톰 히들스턴이 모순적 악역의 연기를 효과적으로 해내어 이 작품의 빌런의 역할을 충실히 해냈다. 그간 단일 작품으로만 보이던 영웅들의 어셈블로 특유 흥미도가 혁신적으로 치솟으며 극적 요소들을 표출한다.\n\t\t\t\t\t\t\t\t\t"
[10] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"

### 여러페이지 크롤링하기. 

In [8]:

base_url <-  "http://movie.daum.net/moviedb/grade?movieId=54081&type=netizen&page="

all.reviews <- c()

for(page in 1:10000){
  url = paste0(base_url,page)
  r = GET(url)
  h = read_html(r)
  comment_area = html_nodes(h, '.list_review')
  comments = html_nodes(comment_area, '.desc_review')
  reviews = html_text(comments)
  
  if(length(reviews) == 0){ break }
  all.reviews = c(all.reviews, reviews)
}

### Save and Read data

In [9]:
all.reviews[1:5]

[1] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                
[2] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                
[3] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"                                                                
[4] " \n\t\t\t\t\t\t\t\t\t\tDC의 저스티스 리그보다 낫지만 다크 나이트 시리즈를 넘을 수 없다.\n\t\t\t\t\t\t\t\t\t"
[5] " \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t"

In [10]:
write.csv(all.reviews,"reviews.csv")

In [11]:
tmp <- read.csv("reviews.csv", stringsAsFactors = F)

In [12]:
head(tmp)

,X,x
1,1,
2,2,
3,3,
4,4,DC의 저스티스 리그보다 낫지만 다크 나이트 시리즈를 넘을 수 없다.
5,5,
6,6,


### 키워드 추출(KoNLP, Stringr)
 - 아직 문제 전체를 이해하는 부분은 없다. 해당 단어만 뽑아서 분석하는 것이 최대한인데. 의외로 많은 부분을 해석할 수 있다. 
 - 전반적으로 하고 싶은 얘기를 알고싶기때문에 단어로도 할 수 있다. 

In [14]:
library(KoNLP) # java로 만들어진 형태소 분석기를 사용한다. 한나눔이라는 형태소 분석기다. 
library(stringr)

In [15]:
extractNoun('이 영화 정말 재미있다')

[1] "영화"   "재미있"

### 형태소 분석
 - 이(관형사) 영화(명사) 정말(부사) 재미있(형용사)다(어미)  
  - 관형사, 어미는 보통 버린다. / 명,형,동사 정도만 분석을 하는데 사용한다.

In [16]:
pos <- paste(SimplePos09('이 영화 정말 재미있다'))
pos

[1] "이/M"     "영화/N"   "정말/M"   "재미있/N" "다/M"

### str_match 찾기

In [17]:
str_match(pos,"N")

NA
N
NA
N
NA


In [18]:
str_match(pos,'[가-힣]+') #[가-힣]한글만 찾아라. 가~힣 // + 1글자 이상 다 찾아라. 

이
영화
정말
재미있
다


In [19]:
str_match(pos,'[가-힣]+/N') #한글로가다가 N으로 끝나는.

NA
영화/N
NA
재미있/N
NA


In [20]:
str_match(pos,'([가-힣]+)/N') 
#한글로가다가 N으로 끝나는 것중에 한글로 된부분을 2번째 필드로 추가

NA,NA
영화/N,영화
NA,NA
재미있/N,재미있
NA,NA


In [21]:
str_match(pos,'([가-힣]+)/[NP]') 
#한글로가다가 N/P으로 끝나는 것중에 한글로 된부분을 2번째 필드로 추가

NA,NA
영화/N,영화
NA,NA
재미있/N,재미있
NA,NA


In [22]:
extracted <- str_match(pos,'([가-힣]+)/[NP]') 
extracted <- extracted[,2]
extracted[!is.na(extracted)]

[1] "영화"   "재미있"

# 한국어 Term Document Matrix 만들기

### Corpus (말뭉치)
 - 기존의 데이터를 합쳐놓은 것
 - 텍스트 분석의 첫번째 단계.
 - 언어학 연구학자들이 말뭉치를 많이 만든다.
 - 사전도 말뭉치를 기반으로 한다. 말뭉치만 가지고는 분석을 할 수 없다.
 - 통계적으로 사용하기 위해선 Term Document Matrix 로 만들어야 한다.


### TM : 어떠한 단어가 나타나는지 표로 만들어주는 것
 - TM 을 바탕으로 한다.
 
### Tips 
 - https://brunch.co.kr/@mapthecity/9
 - 이상한 단어는 제거하도록하고 내가 원하는 단어는 사전에 입력.

In [24]:
library(tm)

In [25]:
texts <- c('hello world','hello text')
texts

[1] "hello world" "hello text"

In [26]:
Corpus(VectorSource(texts))

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 2

In [27]:
cps <- Corpus(VectorSource(texts))
TermDocumentMatrix(cps)

<<TermDocumentMatrix (terms: 3, documents: 2)>>
Non-/sparse entries: 4/2
Sparsity           : 33%
Maximal term length: 5
Weighting          : term frequency (tf)

In [28]:
tdm <- TermDocumentMatrix(cps)
as.matrix(tdm)

,1,2
hello,1,1
world,1,0
text,0,1


### 한국어는 처리를 좀더 해줘야한다. 
 - KoNLP + TM을 잘 해야한다.

In [29]:
texts <- c('오늘은 영화를','영화에는 팝콘')
cps <- Corpus(VectorSource(texts))
tdm <- TermDocumentMatrix(cps)
as.matrix(tdm)
# 우리가 원하는 것은 영화 / 오늘 / 팝콘 이렇게 나오기를 원했는데 안된다.
# KoNLP 처리가 다르게 된 것으로 판단. 

,1,2
영화를,1,0
오늘은,1,0
영화에는,0,1
팝콘,0,1


### 명사만 꺼내는 함수가 있다. extractNoun()

In [30]:
extractNoun('오늘은 영화를')

[1] "오늘" "영화" "를"

In [31]:
ko.words <- function(doc){
  d <- as.character(doc) # doc 를 Character 로 변환 시켜줘야된다. 
  extractNoun(d)
}

In [32]:
options(mc.cores=1) 
# 한나눔 형태소 분석기 Java, TM Packages는 병렬처리를 이용. 두개가 같이 돌면 충돌.

In [33]:
tdm <- TermDocumentMatrix(cps, control=list(tokenize=ko.words)) 
# list로 만들어서 한번에 넘겨주는데 토큰 만드는 것을 ko.words 로
as.matrix(tdm) 
# 결과가 이상하다. 영어의 경우 1자 놈들을 없앤다. 
# 1,2글자 는 없앤다. 왜냐 없기 때문에 한글은 대부분이 1~2글자

,1,2
영화를,1,0
오늘은,1,0
영화에는,0,1
팝콘,0,1


In [34]:
tdm <- TermDocumentMatrix(cps, control=list(tokenize=ko.words, wordLengths=c(1,Inf)))  # 단어길이 제약 풀기.
as.matrix(tdm)  # 팝콘을 추가해야될듯하다.

,1,2
영화를,1,0
오늘은,1,0
영화에는,0,1
팝콘,0,1


### 앞의 내용과 위의 내용 합쳐서 실행

In [35]:
options(mc.cores=1)

In [36]:
all.reviews <- read.csv("reviews.csv",stringsAsFactors=F)$x

In [37]:
review.corpus <- Corpus(VectorSource(all.reviews))
tdm <- TermDocumentMatrix(review.corpus, control = list(tokenize=ko.words,wordLengths=c(1,5)))
tdm.matrix <- as.matrix(tdm)
dim(tdm.matrix)

[1] 1963 1941

In [38]:
row.names(tdm.matrix)[1:10]

[1] "dc의" "넘을" "다크" "수"   "없다" "살짝" "마블" "역시" "여러" "봤음"

### 숫자 지우기
 - removeNumbers=T

In [39]:
tdm <- TermDocumentMatrix(review.corpus, control = list(tokenize=ko.words,wordLengths=c(1,5), removeNumbers=T)) 
tdm.matrix <- as.matrix(tdm)

In [40]:
dim(tdm.matrix)

[1] 1890 1941

In [41]:
rownames(tdm.matrix)[10:20]

[1] "봤음" "그간" "극적" "단일" "이"   "이뤄" "톰"   "특유" "노잼" "면상"
[11] "존나"

### 크롤링 데이터 명/형용사 가지고 오기. 

In [42]:
ko.words2 <- function(doc){
  d <- as.character(doc)
  pos <- paste(SimplePos09(d))
  extracted <- str_match(pos, '([가-힣]+)/[NP]')
  keyword <- extracted[,2]
  return(keyword[!is.na(keyword)])
}

In [43]:
options(mc.cores=1)
tdm <- TermDocumentMatrix(review.corpus, control = list(tokenize=ko.words2,wordLengths=c(1,5), removeNumbers=T)) 
tdm.matrix <- as.matrix(tdm)
dim(tdm.matrix)

[1] 1890 1941

In [44]:
rownames(tdm.matrix)[1:20]

[1] "dc의" "넘을" "다크" "수"   "없다" "살짝" "마블" "역시" "여러" "봤음"
[11] "그간" "극적" "단일" "이"   "이뤄" "톰"   "특유" "노잼" "면상" "존나"

# StopWords 제거하기.
 - 단어를 분석에서 제외하는 방법. 

In [45]:
library(tm)

In [46]:
doc <- c('This is a banana.', 'That is an apple.')

In [47]:
corpus <- Corpus(VectorSource(doc))

In [48]:
TermDocumentMatrix(corpus)

<<TermDocumentMatrix (terms: 4, documents: 2)>>
Non-/sparse entries: 4/4
Sparsity           : 50%
Maximal term length: 6
Weighting          : term frequency (tf)

In [49]:
tdm <- TermDocumentMatrix(corpus) # that, this가 대명사.
as.matrix(tdm)

,1,2
banana,1,0
this,1,0
apple,0,1
that,0,1


In [50]:
tdm <- TermDocumentMatrix(corpus, control = list(removePunctuation = T)) # Punctuation (.과 같은 특수문자 제거.)
as.matrix(tdm)

,1,2
banana,1,0
this,1,0
apple,0,1
that,0,1


In [51]:
tdm <- TermDocumentMatrix(corpus, control = list(removePunctuation = T, wordLengths = c(1,Inf)))  # a, an , is, that, this 제거를 하고싶다.
as.matrix(tdm)

,1,2
a,1,0
banana,1,0
is,1,1
this,1,0
an,0,1
apple,0,1
that,0,1


 - Stopwords를 사용하면 다 없어진다.

In [52]:
tdm <- TermDocumentMatrix(corpus, control = list(removePunctuation = T, wordLengths = c(1,Inf), stopwords = T)) # 영어를 기준으로 제거
as.matrix(tdm)

,1,2
banana,1,0
apple,0,1


In [53]:
stopwords() # 영어

[1] "i"          "me"         "my"         "myself"     "we"        
  [6] "our"        "ours"       "ourselves"  "you"        "your"      
 [11] "yours"      "yourself"   "yourselves" "he"         "him"       
 [16] "his"        "himself"    "she"        "her"        "hers"      
 [21] "herself"    "it"         "its"        "itself"     "they"      
 [26] "them"       "their"      "theirs"     "themselves" "what"      
 [31] "which"      "who"        "whom"       "this"       "that"      
 [36] "these"      "those"      "am"         "is"         "are"       
 [41] "was"        "were"       "be"         "been"       "being"     
 [46] "have"       "has"        "had"        "having"     "do"        
 [51] "does"       "did"        "doing"      "would"      "should"    
 [56] "could"      "ought"      "i'm"        "you're"     "he's"      
 [61] "she's"      "it's"       "we're"      "they're"    "i've"      
 [66] "you've"     "we've"      "they've"    "i'd"        "you'd"     
 [71] "he'd"       "she'd"      "we'd"       "they'd"     "i'll"      
 [76] "you'll"     "he'll"      "she'll"     "we'll"      "they'll"   
 [81] "isn't"      "aren't"     "wasn't"     "weren't"    "hasn't"    
 [86] "haven't"    "hadn't"     "doesn't"    "don't"      "didn't"    
 [91] "won't"      "wouldn't"   "shan't"     "shouldn't"  "can't"     
 [96] "cannot"     "couldn't"   "mustn't"    "let's"      "that's"    
[101] "who's"      "what's"     "here's"     "there's"    "when's"    
[106] "where's"    "why's"      "how's"      "a"          "an"        
[111] "the"        "and"        "but"        "if"         "or"        
[116] "because"    "as"         "until"      "while"      "of"        
[121] "at"         "by"         "for"        "with"       "about"     
[126] "against"    "between"    "into"       "through"    "during"    
[131] "before"     "after"      "above"      "below"      "to"        
[136] "from"       "up"         "down"       "in"         "out"       
[141] "on"         "off"        "over"       "under"      "again"     
[146] "further"    "then"       "once"       "here"       "there"     
[151] "when"       "where"      "why"        "how"        "all"       
[156] "any"        "both"       "each"       "few"        "more"      
[161] "most"       "other"      "some"       "such"       "no"        
[166] "nor"        "not"        "only"       "own"        "same"      
[171] "so"         "than"       "too"        "very"

In [54]:
stopwords('ge') # 독일어

[1] "aber"      "alle"      "allem"     "allen"     "aller"     "alles"    
  [7] "als"       "also"      "am"        "an"        "ander"     "andere"   
 [13] "anderem"   "anderen"   "anderer"   "anderes"   "anderm"    "andern"   
 [19] "anderr"    "anders"    "auch"      "auf"       "aus"       "bei"      
 [25] "bin"       "bis"       "bist"      "da"        "damit"     "dann"     
 [31] "der"       "den"       "des"       "dem"       "die"       "das"      
 [37] "daß"      "derselbe"  "derselben" "denselben" "desselben" "demselben"
 [43] "dieselbe"  "dieselben" "dasselbe"  "dazu"      "dein"      "deine"    
 [49] "deinem"    "deinen"    "deiner"    "deines"    "denn"      "derer"    
 [55] "dessen"    "dich"      "dir"       "du"        "dies"      "diese"    
 [61] "diesem"    "diesen"    "dieser"    "dieses"    "doch"      "dort"     
 [67] "durch"     "ein"       "eine"      "einem"     "einen"     "einer"    
 [73] "eines"     "einig"     "einige"    "einigem"   "einigen"   "einiger"  
 [79] "einiges"   "einmal"    "er"        "ihn"       "ihm"       "es"       
 [85] "etwas"     "euer"      "eure"      "eurem"     "euren"     "eurer"    
 [91] "eures"     "fur"       "gegen"     "gewesen"   "hab"       "habe"     
 [97] "haben"     "hat"       "hatte"     "hatten"    "hier"      "hin"      
[103] "hinter"    "ich"       "mich"      "mir"       "ihr"       "ihre"     
[109] "ihrem"     "ihren"     "ihrer"     "ihres"     "euch"      "im"       
[115] "in"        "indem"     "ins"       "ist"       "jede"      "jedem"    
[121] "jeden"     "jeder"     "jedes"     "jene"      "jenem"     "jenen"    
[127] "jener"     "jenes"     "jetzt"     "kann"      "kein"      "keine"    
[133] "keinem"    "keinen"    "keiner"    "keines"    "konnen"    "konnte"   
[139] "machen"    "man"       "manche"    "manchem"   "manchen"   "mancher"  
[145] "manches"   "mein"      "meine"     "meinem"    "meinen"    "meiner"   
[151] "meines"    "mit"       "muss"      "musste"    "nach"      "nicht"    
[157] "nichts"    "noch"      "nun"       "nur"       "ob"        "oder"     
[163] "ohne"      "sehr"      "sein"      "seine"     "seinem"    "seinen"   
[169] "seiner"    "seines"    "selbst"    "sich"      "sie"       "ihnen"    
[175] "sind"      "so"        "solche"    "solchem"   "solchen"   "solcher"  
[181] "solches"   "soll"      "sollte"    "sondern"   "sonst"     "uber"     
[187] "um"        "und"       "uns"       "unse"      "unsem"     "unsen"    
[193] "unser"     "unses"     "unter"     "viel"      "vom"       "von"      
[199] "vor"       "wahrend"   "war"       "waren"     "warst"     "was"      
[205] "weg"       "weil"      "weiter"    "welche"    "welchem"   "welchen"  
[211] "welcher"   "welches"   "wenn"      "werde"     "werden"    "wie"      
[217] "wieder"    "will"      "wir"       "wird"      "wirst"     "wo"       
[223] "wollen"    "wollte"    "wurde"     "wurden"    "zu"        "zum"      
[229] "zur"       "zwar"      "zwischen"

In [55]:
stopwords('fr') # 프랑스어

[1] "au"       "aux"      "avec"     "ce"       "ces"      "dans"    
  [7] "de"       "des"      "du"       "elle"     "en"       "et"      
 [13] "eux"      "il"       "je"       "la"       "le"       "leur"    
 [19] "lui"      "ma"       "mais"     "me"       "meme"     "mes"     
 [25] "moi"      "mon"      "ne"       "nos"      "notre"    "nous"    
 [31] "on"       "ou"       "par"      "pas"      "pour"     "qu"      
 [37] "que"      "qui"      "sa"       "se"       "ses"      "son"     
 [43] "sur"      "ta"       "te"       "tes"      "toi"      "ton"     
 [49] "tu"       "un"       "une"      "vos"      "votre"    "vous"    
 [55] "c"        "d"        "j"        "l"        "a"        "m"       
 [61] "n"        "s"        "t"        "y"        "ete"      "etee"    
 [67] "etees"    "etes"     "etant"    "suis"     "es"       "est"     
 [73] "sommes"   "etes"     "sont"     "serai"    "seras"    "sera"    
 [79] "serons"   "serez"    "seront"   "serais"   "serait"   "serions" 
 [85] "seriez"   "seraient" "etais"    "etait"    "etions"   "etiez"   
 [91] "etaient"  "fus"      "fut"      "fumes"    "futes"    "furent"  
 [97] "sois"     "soit"     "soyons"   "soyez"    "soient"   "fusse"   
[103] "fusses"   "fut"      "fussions" "fussiez"  "fussent"  "ayant"   
[109] "eu"       "eue"      "eues"     "eus"      "ai"       "as"      
[115] "avons"    "avez"     "ont"      "aurai"    "auras"    "aura"    
[121] "aurons"   "aurez"    "auront"   "aurais"   "aurait"   "aurions" 
[127] "auriez"   "auraient" "avais"    "avait"    "avions"   "aviez"   
[133] "avaient"  "eut"      "eumes"    "eutes"    "eurent"   "aie"     
[139] "aies"     "ait"      "ayons"    "ayez"     "aient"    "eusse"   
[145] "eusses"   "eut"      "eussions" "eussiez"  "eussent"  "ceci"    
[151] "cela"     "cela"     "cet"      "cette"    "ici"      "ils"     
[157] "les"      "leurs"    "quel"     "quels"    "quelle"   "quelles" 
[163] "sans"     "soi"

- 한국어는 없다.... 직접만들어야 한다. 

In [56]:
tdm <- TermDocumentMatrix(corpus, control = list(removePunctuation = T, wordLengths = c(1,Inf), stopwords = stopwords('ge')))
# 독일어로 없앤다. 
as.matrix(tdm)

,1,2
a,1,0
banana,1,0
is,1,1
this,1,0
apple,0,1
that,0,1


In [57]:
tdm <- TermDocumentMatrix(corpus, control = list(removePunctuation = T, wordLengths = c(1,Inf), stopwords = c('a','an','is'))) 
as.matrix(tdm)

,1,2
banana,1,0
this,1,0
apple,0,1
that,0,1


 - stopwords 목록에 제거할 단어를 작성하여 넣어주면 된다. csv파일로 작성하여 불러오면 된다. 

# 14. Co-occurrence Matrix 만들기 
 - 두 단어가 얼마나 함께 나타나는지 

In [58]:
texts

[1] "오늘은 영화를" "영화에는 팝콘"

In [59]:
options(mc.cores=1)

In [60]:
cps <- Corpus(VectorSource(texts))

In [61]:
tdm <- TermDocumentMatrix(cps, control = list(tokenize=ko.words,wordLengths=c(1,Inf)))

In [62]:
tdm.matrix <- as.matrix(tdm)

In [63]:
tdm.matrix  %*% t(tdm.matrix ) # Co-occurrence Matrix 만들기. 

,영화를,오늘은,영화에는,팝콘
영화를,1,1,0,0
오늘은,1,1,0,0
영화에는,0,0,1,1
팝콘,0,0,1,1


 - %*% : 행렬의 곱. 
 - T : Transpose 

### 크롤링 데이터로 해보기 

In [64]:
all.views <- read.csv('reviews.csv', stringsAsFactors=F)$x

In [65]:
review.corpus <- Corpus(VectorSource(all.views))

 - weighting=weightBin  단어들이 있나 없나만 보면된다. 여러번 나왔다는거는 의미가 없다. (Co-occurence Matrix에서는)

In [66]:
tdm <- TermDocumentMatrix(review.corpus, 
                          control = list(tokenize=ko.words,wordLengths=c(2,5),
                                         removeNumbers=T, 
                                         weighting=weightBin,
                                         removePunctuation=T))

In [67]:
tdm.matrix <- as.matrix(tdm)

In [68]:
dim(tdm.matrix)

[1] 1882 1941

### 다빈도 단어 추출.

In [69]:
word.count <- rowSums(tdm.matrix)
word.order <- order(word.count,decreasing = T)

In [70]:
word.order[1:20]

[1]  78  57  51  49 124 175 284 152  59 183  44 119  14  37   8  26 350  67   4
[20] 353

In [71]:
rownames(tdm.matrix)[word.order[1:20]]

[1] "영화" "정말" "너무" "그냥" "진짜" "좀"   "헐크" "ㅋㅋ" "더"   "완전"
[11] "잘"   "최고" "이"   "다"   "역시" "액션" "점"   "보고" "수"   "ㅋ"

In [72]:
freq.words <- tdm.matrix[word.order[1:20],]

In [73]:
dim(freq.words)

[1]   20 1941

In [74]:
cooccur <- freq.words %*% t(freq.words)

In [75]:
cooccur

,영화,정말,너무,그냥,진짜,좀,헐크,ㅋㅋ,더,완전,잘,최고,이,다,역시,액션,점,보고,수,ㅋ
영화,235,26,18,15,12,9,10,12,18,10,22,5,16,12,7,10,13,10,16,4
정말,26,148,16,8,12,11,11,6,10,9,11,6,7,9,9,6,10,7,8,2
너무,18,16,126,15,12,5,5,10,9,9,11,4,2,8,6,5,4,3,3,4
그냥,15,8,15,94,2,9,8,6,4,6,5,3,4,6,1,6,6,1,3,2
진짜,12,12,12,2,94,7,2,5,10,7,6,6,4,8,4,2,3,8,2,1
좀,9,11,5,9,7,91,7,3,6,3,4,1,2,3,3,9,7,8,2,5
헐크,10,11,5,8,2,7,91,13,4,6,6,7,7,5,1,3,3,5,2,2
ㅋㅋ,12,6,10,6,5,3,13,85,4,6,3,6,5,5,3,5,1,2,2,5
더,18,10,9,4,10,6,4,4,73,1,3,1,6,7,3,1,2,10,3,2
완전,10,9,9,6,7,3,6,6,1,73,3,5,1,4,3,2,4,6,1,7


In [76]:
library(qgraph)

Warning message:
: package 'qgraph' was built under R version 3.3.2

 - 대각선의 위치에 있는 숫자들이 나온 값이다. diag()가 가운데 값만 보낸다. 

In [ ]:
qgraph(cooccur,
       labels=rownames(cooccur), # 가로세로 모두 단어다 그래서 colnames로 해도상관없다. 단어가 안짤리고 잘나온다. 
       diag=F, # 자기자신을 나타내는 대각선 부분을 없앤다. 
       layout='spring', # Node의 위치를 바꿀수있다. 
       edge.color='blue',
       vsize=log(diag(cooccur)*3),) # edge : 선, V : 노드 

<img src="Aven.PNG">